In [2]:
import pandas as pd #data processing
import numpy as np #maths
import requests as req #get data from url
import gzip
from io import BytesIO
import sklearn



In [3]:
#get film datasets

#set urls
url_title_basics = 'https://datasets.imdbws.com/title.basics.tsv.gz' #film name, year, runtime, genres
url_crew = 'https://datasets.imdbws.com/title.crew.tsv.gz' #writers and directors
url_actors = 'https://datasets.imdbws.com/title.principals.tsv.gz' 

#download from url
res_title_basics = req.get(url_title_basics).content
res_crew = req.get(url_crew).content
res_actors = req.get(url_actors).content

#decompress
title_basics_gzip = gzip.decompress(res_title_basics)
crew_basics_gzip = gzip.decompress(res_crew)
title_actors_gzip = gzip.decompress(res_actors)

#read into array
titles = pd.read_csv(BytesIO(title_basics_gzip), delimiter='\t')
crews = pd.read_csv(BytesIO(crew_basics_gzip), delimiter='\t')
actors = pd.read_csv(BytesIO(title_actors_gzip), delimiter='\t')



/var/folders/14/hwhy2lhx06d4s8wj8trw2n0r0000gn/T/ipykernel_67045/1935632158.py:19: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles = pd.read_csv(BytesIO(title_basics_gzip), delimiter='\t')


In [11]:
#clean data

#remove non-movie rows
titles = titles[titles['titleType'] == 'movie']

#get tconsts for remaining non-movie rows
film_tconsts = titles['tconst'].tolist()

#remove corresponding non-movie rows
crews = crews[crews['tconst'].isin(film_tconsts)]
actors = actors[actors['tconst'].isin(film_tconsts)]

#set columns to remove from dataset
remove_from_titles = ['originalTitle', 'endYear']
remove_from_actors = ['ordering','job','characters']

titles = titles.drop(columns=remove_from_titles)
actors = actors.drop(columns=remove_from_actors)


In [21]:
titles.head()


,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,0,1894,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,0,1897,100,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,0,1905,100,\N
570,tt0000574,movie,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,0,1907,90,Drama
